# Data exploration

The key concept of exploratory data analysis (EDA) or visual exploration of data is to understand the relationships in the data set. Specifically, by using visualization when you approach a new data set you can:

- Explore complex data sets, using visualization to develop understanding of the inherent relationships
- Use different chart types to create multiple views of data to highlight different aspects of the inherent relationships
- Use plot aesthetics to project multiple dimensions 
- Apply conditioning or faceting methods to project multiple dimensions

This notebook contains an introduction to the methods data scientists use to prepare and explore a data set. Data scientists spend a lot of time manipulating data to clean and prepare it (**data munging**) and to understand the relationships within the variables in the data (**data exploration**). Visualization is a key skill for any data scientist performing these tasks. 

Before performing any type of inference or machine learning on a data set, you should develop an understanding of the relationships in the data set. Skipping this step can lead to wasted time from unexpected problems when building models, or in constructing models with poor performance. When starting any data science project, it is a good idea to plan on spending considerable time exploring the data. These exploration steps are often performed in conjunction with data cleaning and preparing, as visualization often highlights problems with data.

In [ ]:
# import sys
# !{sys.executable} -m pip install statsmodels==0.13.2
# !{sys.executable} -m pip install mlxtend==0.21.0
# !{sys.executable} -m pip install graphviz==0.20.1

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import itertools

sns.set(rc = {'figure.figsize': (10, 8)})

### About the data set

This data set is from the [Canadian Open Government website](https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64/resource/87fc1b5e-fafc-4d44-ac52-66656fc2a245#additional-info)  and the [Canadian Association of Road Safety Professionals](https://www.carsp.ca/research/resources/canadian-vehicle-specifications/).

##### Columns
- **make** - vehicle make
- **short_model_name** - short model name
- **overall_length_cm, overall_width_cm , overall_height_cm, wheelbase_cm** - dimensions of vehicle in cm
- **curb_weight_kg** - weight of vehicle
- **weight_distribution_pct_front** -percent of the vehicle weight supported by the front axle
- **vehicle_class** - type of vehicle
- **engine_size_l** - engine size in liters
- **cylinders** number of engine cylinders
- **transmission** - manual or various types of automatic transmission
- **fuel** - regular gas, premium_gas or diesel
- **fuel_consumption_mpg** - miles per gallon of fuel for combined ratings (55% city, 45% hwy)
- **smog** - tailpipe emissions of smog-forming pollutants rated on a scale from 1 (worst) to 10 (best)
- **full_model_name** - full vehicle description

## Load and examine the data

Let's get started. The function shown in the cell below loads the data from the .csv file. Some minimal clean up is performed on the data. Rows with missing values are dropped and some columns are converted from strings containing numbers to numeric data. The result is a Pandas data frame.

Execute the code in this cell to load the data into your notebook. **Make sure the canadian_cars_2022.csv file path is correct from your working directory!**

In [ ]:
def read_auto_data(file = "../data/canadian_cars_2022.csv"):
    'Function to load the auto data set from a .csv file' 
    import pandas as pd
    import numpy as np

    ## Read the .csv file with the pandas read_csv method
    df = pd.read_csv(file)
    
    ## Split the number of gears from the type of transmission, decode fuel
    df['gears'] = df['transmission'].str.extract(r'([0-9]+)').astype('Int64')
    df['gears'] = df['gears'].fillna(1) # "gearless" continuously_variable vehicles
    df['fuel'].replace({'X': 'regular_gas', 
                             'Z': 'premium_gas', 
                             'D': 'diesel'}, inplace = True)
    df['transmission'] = df['transmission'].str.extract(r'([A-Z]+)')
    df['transmission'].replace({'A': 'automatic', 
                             'AM': 'automated_manual', 
                             'AS': 'automatic_select_shift', 
                             'AV': 'continuously_variable', 
                             'M': 'manual'}, inplace = True)
    
    ## Remove rows with missing values
    df = df.dropna(axis = 0).reset_index(drop= True)
    return df


auto_df = read_auto_data()

Next, have a first look at the dimensions and data types of the column of the data frame.

In [ ]:
auto_df.info()

Some key things to notice here are:

- There are 210 rows or cases in the data set.
- The data set has 16 variables or columns.
- Columns have a type of either `object` (character), `float64` (floating point number), or `int64` (integer). 


## Basic summary statistics

You can also learn a lot about a data set by looking at statistical summaries. The Pandas `describe` method does just this. Execute the code in the cell below and note the result.

In [ ]:
with pd.option_context('display.float_format', '{:0.2f}'.format):
    display(auto_df.describe())

Notice, that summary statistics are only provided for numeric columns. For these column this summary provides some useful information:

- The minimum and maximum values of each variable are displayed.
- The quantiles are shown; 25%, 50% (median), 75%. Note that, for several of these variables, the upper interquartile ranges (IQR) are different from the lower IQR, indicating the distributions are skewed. 
- The mean is displayed. Compare the mean and the median, noting that they can be quite different, like in the case of cylinders. This is a sign of a skewed distribution. 
- The standard deviation gives a measure of the dispersion of the values.

We can also obtain relevant summary statistics for categorical columns using the `include = 'object'`, which limits the columns to those of type `object`.

In [ ]:
auto_df.describe(include = 'object')

Next, let's look at the first and last few lines of the data frame using the `head` and `tail` methods.

In [ ]:
auto_df.head(6)

In [ ]:
auto_df.tail(6)

### Exercise

Say we wanted to examine the `overall_length_cm` column in the data. We already saw that we can use `describe`.

In [ ]:
auto_df[['overall_length_cm']].describe()

Using the `seaborn` library, we can plot a line plot of `length` against the index of the data `auto_df.index`:

In [ ]:
sns.lineplot(x = auto_df.index, 
             y = 'overall_length_cm', 
             data = auto_df
            ).set(xlabel = 'index', 
                  ylabel = 'length');

- Do you find the above visualization useful? When can it be useful and when not?

- Sort the data by `overall_length_cm` and reset the index of the data, then recreate the above plot. Do you find the new plot useful?

In [ ]:
## your code goes here

- Run the `nlargest` method to show the top 10 values for `overall_length_cm`. Can you show these values on the plot above?

In [ ]:
## your code goes here

If the data is really big, instead of plotting `overall_length_cm` itself, we can use the `quantile` method to obtain percentiles for `overall_length_cm` and plot those instead. 

- Use `quantile` to obtain the 0th, 1st, 2nd, 3rd, ..., 100th percentile for `overall_length_cm` and plot the percentiles. HINT: Use `np.arange(0, 1.01, 0.01)` to get the percentiles all at once.

In [ ]:
## your code goes here

- Do the results look similar to what we saw earlier?

### End of exercise

## Frequency tables

The `groupby` method can be used to get separate summary statistics for each category in a categorical column. The `agg` or aggregation method can be used to specify which summary statistics we want. In this case, we will use the `count` operator to get counts of rows in each category of the variable. 

In [ ]:
auto_df.groupby(['make']
      ).agg(
            model_count = ('short_model_name', 'count')
          ).head()

We can look at frequencies of a combinations of two categorical variables, such combinations are also known as **interactions**. The groups are organized by the order of the columns specified.

In [ ]:
auto_df.groupby(['make', 'fuel']
      ).agg(
            model_count = ('short_model_name', 'count')
           ).head()

The above table gives us a feel for which manufactures only make gas cars and which ones make both gas and diesel cars, and how many of each. Notice that groups with counts of zero do not display in the table. 

When we look at interactions of two columns, the result is also called a **two-way table**. Similarly, we can have **three-way tables**, and so on.

In [ ]:
auto_df.groupby(['make', 'fuel', 'cylinders']
      ).agg(
            model_count = ('short_model_name', 'count')
           ).head()

Notice that with three levels the table takes a bit more effort to examine. As the number of grouping levels increases, frequency tables become progressively more difficult to understand. 

### Exercise

- Create and execute the code to create a frequency table of cars by `vehicle_class` and `cylinders`.

In [ ]:
## your code goes here

- Which vehicle class and cylinder engine configuration is the most frequent?
- Ignoring missing values, which vehicle class and cylinder engine configurations are the least frequent?
- What happens to the results if we add the `unstack()` method to the end?

In [ ]:
## your code goes here

- Create the same frequency table using [pd.crosstab](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html) instead. What is the advantage of using `pd.crosstab`? What are the `normalize` and `margin` arguments used for?

In [ ]:
## your code goes here

Visualize the result of `pd.crosstabs` above, using `sns.heatmap`. Decide if you should normalize and by which dimension. You can change the color map using the [cmap parameter](https://matplotlib.org/stable/tutorials/colors/colormaps.html). For example, try cmap = 'cividis' and see how the visualization changes.

In [ ]:
## your code goes here

### End of exercise

## Basic univariate charts

In this notebook we work with a number of common univariate chart types. By univariate we mean that it is a summary of only a single variable.

- **bar plots**
- **histograms**
- **box plots**
- **line plots**

### Bar plots

To start our exploration of these data we will work with bar plots. Bar plots are used to display the counts or frequency of unique values of a categorical variable. The length of the bar represents the count for each unique category of the variable.

In [ ]:
ax = sns.countplot(y = 'vehicle_class', 
              hue = 'fuel', 
              data = auto_df)
ax.legend(loc = 'right');

From this display you can see that the most frequent body type is the small SUV and the least frequent is the special purpose vehicle, and diesels make up a small percentage of cars overall.

### Exercise

It would be easier to understand this plot if the frequencies of the categories were ordered. This would be particularly true if there were a large number of categories or categories with similar counts. 

- Use the `value_counts()` method to get a list where each `body-style` is ordered by frequency, then use the `order` argument in `countplot` to reorder the data.

In [ ]:
## your code goes here

- Compare the relationship between the number of gears and the style of transmission using one of the above techniques.
- What types of tranmissions are most common? 
- How does the type of transmission for the vehicle with only one gear impact your opinion on if this data point is correct?

In [ ]:
## your code goes here

### End of exercise

The examples above use `countplot`, but we can also use the more general `catplot` function to create bar plots. 

In [ ]:
g = sns.catplot(x = 'make', 
                data = auto_df, 
                kind = 'count', 
                color = 'lightblue', 
                aspect = 2)
g.set_xticklabels(rotation = 90);

### Histograms

Histograms are related to bar plots. Histograms are used for numeric variables. Whereas, a bar plot shows the counts of unique categories, a histogram shows the number of data with values in each bin. The bins divide the values of the variable into equal segments. The vertical axis of the histogram shows the count of data values within each bin.  

In [ ]:
sns.histplot(auto_df['curb_weight_kg'], kde = False);

The histogram shows that the weight of automobiles are roughly normally distributed, with a cluster of much heavier vehicles.

### Box plots

Box plots, also known as box and whisker plots, were introduced by John Tukey in 1970. Box plots are another way to visualize the distribution of data values. In this respect, box plots are comparable to histograms, but are quite different in presentation.

In [ ]:
auto_df[['curb_weight_kg']].describe()

In [ ]:
ax = sns.boxplot(x = 'curb_weight_kg', 
                 data = auto_df);

Compare the above plot to the histogram of weight. You can see from both plots that the distribution of weight is fairly symmetric. Further, there are a number of heavy cars that appear as outliers. 

The above plot is quite basic. Most importantly, this plot lacks the annotation required for someone to quickly understand it. At the minimum a title and an axis label are required. Fortunately, adding these attributes to the plot is relatively easy, since matplotlib provides methods to do so. 

There is another issue with this plot. Notice that the limits of the x axis does not include `0`, which can distort the observer's view of the data. This is particularly the case when making comparisons between variables, of subsets of variables. So we use the `set_xlim` method to reset the axis.

### Exercise

Box plots can be especially helpful when comparing different categories. 

- Display a box plot of auto weight grouped by both `fuel` and `gears`.

In [ ]:
## your code goes here

- Are diesel cars generally heavier than vehicles using gas? 
- What type of vehicles tend to be the lightest?
- Which type of car shows the greatest weight range? The most outliers?

### End of exercise

# Assignment

In this assignment, we look at some examples of third-party packages for EDA and visualizaiton. The data that we use is time series data. Time series data is very common and dealing with time series data sometimes requires that we take special care. The time series we use in this assignment has the monthly number of subspots between 1749 and 2013. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc = {'figure.figsize': (8, 8)})
sunspots = pd.read_csv('../data/sunspots.csv')

- Convert the timestamp column into `datetime` and then set it as the index of the data. This is how we can let pandas know that we have a time series data. <span style="color:red" float:right>[2 point]</span>

In [ ]:
## your code goes here

- Create a line plot of this time series, using the index in the x-axis. <span style="color:red" float:right>[1 point]</span>

In [ ]:
## your code goes here

- The code at the beginning of the assignment set the figure size to a square aspect ratio of 8 by 8. Modify that code so the plot has a more pleasing wide aspect ratio. <span style="color:red" float:right>[1 point]</span>

In [ ]:
## your code goes here

It is often helpful to smooth out a time series plot using a moving average. In `pandas`, we can easily calculate moving averages (or other statistic) using the `rolling` method. An example is shown below.

In [ ]:
x = pd.Series([5, 9, 10, 4, 8])
x.rolling(3).mean()

- State why the first two values of series above are `NaN`. <span style="color:red" float:right>[2 point]</span>

In [ ]:
## your answer goes here (no code required)

- Create a new plot that shows both the original time series and the smoothed out time series using a rolling average with the window size set to 24. Your plot should include labels to indicate which line plot is which, such as 'original' and 'rolling average'. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## your code goes here

Another way to smooth out the data is to aggregate the data from monthly to yearly. There are different ways of doing that, but the easiest way is by using the `resample` method, as shown [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html). Since we are going from monthly data to yearly, we call this **downsampling**. We can also use `resample` to **upsample** the data.

- Use resample to aggregate the data to yearly totals, then plot a line plot of the data. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## your code goes here

The plot should show the number of sunspots on the vertical axis and the datetime on the horizontal axis. 

# End of assignment